In [ ]:
!pip install torch_geometric

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [ ]:
%cd hackaton/

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [ ]:
!ls -lh datasets

In [ ]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


In [ ]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [ ]:
def train(
    data_loader,
    model,
    optimizer,
    scheduler,
    criterion,
    device,
    save_checkpoints=False,
    checkpoint_path="ckpt",
    current_epoch=0,
    grad_clip: float = 2.0,
):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for data in tqdm(data_loader, desc="Train-batches", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss   = criterion(output, data.y)
        loss.backward()

        # Gradient-clipping per stabilità
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total   += data.y.size(0)

    scheduler.step()

    # Eventuale checkpoint
    if save_checkpoints:
        fname = f"{checkpoint_path}_epoch_{current_epoch+1}.pth"
        torch.save(model.state_dict(), fname)
        print(f"✓ Checkpoint salvato: {fname}")

    return total_loss / len(data_loader), correct / total

In [ ]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [ ]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [ ]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [ ]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [ ]:
def get_arguments():
    args = {}
    #args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['train_path'] = "/kaggle/working/hackaton/datasets/A/train.json.gz"
    #args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['test_path'] = "/kaggle/working/hackaton/datasets/A/test.json.gz"
    #args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['num_checkpoints'] = 3
    #args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['drop_ratio'] = 0.1
    #args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['batch_size'] = 64
    #args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['epochs'] = 10
    #args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    #args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [ ]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

In [ ]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [ ]:
class SymmetricCrossEntropy(nn.Module):
    """
    Robust loss (Ghosh et al. 2017) contro label rumorose.
    L = α · CE(p, q)  +  β · CE(q, p)
    """
    def __init__(self, num_classes: int, alpha: float = 0.7, beta: float = 1.3):
        super().__init__()
        self.alpha = alpha
        self.beta  = beta
        self.ce    = nn.CrossEntropyLoss()
        self.num_classes = num_classes

    def forward(self, logits, target):
        ce = self.ce(logits, target)

        # Reverse CE  (model distribuz. → label one-hot)
        pred = torch.softmax(logits, dim=1).clamp(min=1e-7, max=1.0)
        target_one_hot = torch.zeros_like(pred).scatter_(1, target.view(-1,1), 1)
        rce = (- (target_one_hot * torch.log(pred)).sum(dim=1)).mean()

        return self.alpha * ce + self.beta * rce

In [ ]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, JumpingKnowledge, GlobalAttention
from torch_geometric.utils import dropout_edge, add_self_loops

class GraphSAGEClassifier(nn.Module):
    """
    3 layer GraphSAGE con:
      • aggregatore 'max'
      • BatchNorm + Dropout dopo ogni layer
      • Jumping-Knowledge ('cat') per evitare oversmoothing
      • Global Attention Pooling per pesare i nodi rilevanti
    Usa un embedding fittizio (featureless) come nella tua versione.
    """
    def __init__(
        self,
        input_dim:  int = 128,
        hidden_dim: int = 128,
        output_dim: int = 6,
        num_layers: int = 3,
        dropout:    float = 0.3,
        jk_mode:    str   = "cat",
    ):
        super().__init__()

        # Featureless embedding (1 categoria di nodi)
        self.embedding = nn.Embedding(1, input_dim)

        # ── GraphSAGE stack ────────────────────────────────────────────────────
        self.convs, self.bns = nn.ModuleList(), nn.ModuleList()
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            self.convs.append(SAGEConv(in_dim, hidden_dim, aggr="max"))
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        # Jumping-Knowledge per combinare i layer
        self.jump = JumpingKnowledge(jk_mode)
        jk_out_dim = hidden_dim * num_layers if jk_mode == "cat" else hidden_dim

        # Global Attention pooling (gate = MLP a 2 layer)
        self.pool = GlobalAttention(
            gate_nn = nn.Sequential(
                nn.Linear(jk_out_dim, jk_out_dim // 2),
                nn.ReLU(),
                nn.Linear(jk_out_dim // 2, 1)
            )
        )

        self.drop = nn.Dropout(dropout)
        self.fc   = nn.Linear(jk_out_dim, output_dim)

    # ─────────────────────────────────────────────────────────────────────────
    def forward(self, data):
        # (1) self-loops & edge-dropout on-the-fly
        edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
        edge_index, _ = dropout_edge(edge_index, p=0.2, training=self.training)

        # (2) node embedding
        x = self.embedding(data.x.squeeze())  # (N, input_dim)

        # (3) GraphSAGE layers
        xs = []
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index)
            x = bn(x)
            x = torch.relu(x)
            x = self.drop(x)
            xs.append(x)

        # (4) Jumping-Knowledge aggregation
        x = self.jump(xs)                     # (N, jk_out_dim)

        # (5) Graph-level representation
        x = self.pool(x, data.batch)          # (B, jk_out_dim)
        return self.fc(x)


In [ ]:
script_dir = os.getcwd() 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
model = GraphSAGEClassifier().to(device)

optimizer  = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-4)
scheduler  = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=num_epochs, eta_min=1e-5
)
criterion  = SymmetricCrossEntropy(num_classes=6).to(device)

In [ ]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [ ]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [ ]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)
 
    print(num_classes = len(set(d.y.item() for d in train_dataset)))

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)